In [ ]:
ATTACK_EPS = 0.5
ATTACK_STEPSIZE = 0.1
ATTACK_STEPS = 10
NUM_WORKERS = 8
BATCH_SIZE = 10

## Set up dataset

In [ ]:
import torch as ch
from robustness.datasets import CIFAR
ds = CIFAR('/tmp')

## Instantiate model

In [ ]:
from robustness.model_utils import make_and_restore_model
model, _ = make_and_restore_model(arch='resnet50', dataset=ds,
             resume_path='/data/theory/robustopt/robust_models/cifar_l2_eps_05/checkpoint.pt.best')
model.eval()
pass

## Set up loaders

In [ ]:
_, test_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)
_, (im, label) = next(enumerate(test_loader))

## Generating untargeted adversarial examples

In [ ]:
kwargs = {
    'constraint':'2', # use L2-PGD
    'eps': ATTACK_EPS, # L2 radius around original image
    'step_size': ATTACK_STEPSIZE,
    'iterations': ATTACK_STEPS,
    'do_tqdm': True,
}

In [ ]:
_, im_adv = model(im, label, make_adv=True, **kwargs)

In [ ]:
from robustness.tools.vis_tools import show_image_row
from robustness.tools.label_maps import CLASS_DICT

# Get predicted labels for adversarial examples
pred, _ = model(im_adv)
label_pred = ch.argmax(pred, dim=1)

# Visualize test set images, along with corresponding adversarial examples
show_image_row([im.cpu(), im_adv.cpu()],
         tlist=[[CLASS_DICT['CIFAR'][int(t)] for t in l] for l in [label, label_pred]],
         fontsize=18,
         filename='./adversarial_example_CIFAR.png')

## Targeted adversarial examples

In [ ]:
kwargs = {
    'constraint':'2',
    'eps': ATTACK_EPS,
    'step_size': ATTACK_STEPSIZE,
    'iterations': ATTACK_STEPS,
    'targeted': True,
    'do_tqdm': True
}

In [ ]:
targ = ch.zeros_like(label)

In [ ]:
_, im_adv = model(im, targ, make_adv=True, **kwargs)

In [ ]:
# Visualize test set images, along with corresponding adversarial examples
show_image_row([im.cpu(), im_adv.cpu()],
         tlist=[[CLASS_DICT['CIFAR'][int(t)] for t in l] for l in [label, label_pred]],
         fontsize=18,
         filename='./adversarial_example_CIFAR.png')

## Custom Input Manipulation (Representation Inversion)

In [ ]:
_, (im_inv, label_inv) = next(enumerate(test_loader)) # Images to invert
with ch.no_grad():
    (_, rep_inv), _ = model(im_inv, with_latent=True) # Corresponding representation

In [ ]:
def inversion_loss(model, inp, targ):
    # Compute representation for the input
    _, rep = model(inp, with_latent=True, fake_relu=True)
    # Normalized L2 error w.r.t. the target representation
    loss = ch.div(ch.norm(rep - targ, dim=1), ch.norm(targ, dim=1))
    return loss, None

In [ ]:
kwargs = {
    'custom_loss': inversion_loss,
    'constraint':'2',
    'eps': 1000,
    'step_size': 1,
    'iterations': 1000,
    'targeted': True,
    'do_tqdm': True,
}

In [ ]:
im_seed = ch.clamp(ch.randn_like(im_inv) / 20 + 0.5, 0, 1)

In [ ]:
_, im_matched = model(im_seed, rep_inv, make_adv=True, **kwargs)

In [ ]:
show_image_row([im_inv.cpu(), im_seed.cpu(), im_matched.cpu()],
         ["Original", r"Seed ($x_0$)", "Result"],
         fontsize=18,
         filename='./custom_inversion_CIFAR.png')

## Other optimization methods (example: Fourier basis)

In [ ]:
kwargs = {
  'custom_loss': inversion_loss,
  'constraint':'fourier',
  'eps': 1000, # ignored anyways
  'step_size':  500, # have to re-tune LR
  'iterations': 10000,
  'targeted': True,
  'do_tqdm': True,
}

In [ ]:
im_seed = ch.randn(BATCH_SIZE, 3, 32, 32, 2) / 5

In [ ]:
_, im_matched = model(im_seed, rep_inv, make_adv=True, **kwargs)

In [ ]:
show_image_row([im_inv.cpu(), im_matched.cpu()],
         ["Original", "Result"],
         fontsize=18,
         filename='./custom_inversion_CIFAR_fourier.png')